In [ ]:
pip install jinja2


In [ ]:
import string
from hashlib import sha1
from datetime import datetime
import pandas as pd
from jinja2 import Environment, FileSystemLoader,Template
## Change below siteinfo if different website is picked ##
tewiki = '''<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="te">
	<siteinfo>
		<sitename>tewiki</sitename>
		<dbname>indicwiki</dbname>
		<base>https://tewiki.iiit.ac.in/index.php/%E0%B0%AE%E0%B1%8A%E0%B0%A6%E0%B0%9F%E0%B0%BF_%E0%B0%AA%E0%B1%87%E0%B0%9C%E0%B1%80</base>
		<generator>MediaWiki 1.34.0</generator>
		<case>first-letter</case>
		<namespaces>
			<namespace key="-2" case="first-letter">మీడియా</namespace>
			<namespace key="-1" case="first-letter">ప్రత్యేక</namespace>
			<namespace key="0" case="first-letter" />
			<namespace key="1" case="first-letter">చర్చ</namespace>
			<namespace key="2" case="first-letter">వాడుకరి</namespace>
			<namespace key="3" case="first-letter">వాడుకరి చర్చ</namespace>
			<namespace key="4" case="first-letter">Project</namespace>
			<namespace key="5" case="first-letter">Project చర్చ</namespace>
			<namespace key="6" case="first-letter">దస్త్రం</namespace>
			<namespace key="7" case="first-letter">దస్త్రంపై చర్చ</namespace>
			<namespace key="8" case="first-letter">మీడియావికీ</namespace>
			<namespace key="9" case="first-letter">మీడియావికీ చర్చ</namespace>
			<namespace key="10" case="first-letter">మూస</namespace>
			<namespace key="11" case="first-letter">మూస చర్చ</namespace>
			<namespace key="12" case="first-letter">సహాయం</namespace>
			<namespace key="13" case="first-letter">సహాయం చర్చ</namespace>
			<namespace key="14" case="first-letter">వర్గం</namespace>
			<namespace key="15" case="first-letter">వర్గం చర్చ</namespace>
			<namespace key="828" case="first-letter">మాడ్యూల్</namespace>
			<namespace key="829" case="first-letter">మాడ్యూల్ చర్చ</namespace>
			<namespace key="2300" case="first-letter">Gadget</namespace>
			<namespace key="2301" case="first-letter">Gadget talk</namespace>
			<namespace key="2302" case="case-sensitive">Gadget definition</namespace>
			<namespace key="2303" case="case-sensitive">Gadget definition talk</namespace>
			<namespace key="2600" case="first-letter">Topic</namespace>
		</namespaces>
	</siteinfo>\n'''

# 300000 - 350000 => school's articles
# Global Variables
page_id = 1500001

user_id = "19051"
username ="Srihitha"

# Funtions to write page to file object
def sha36(page_id):
	page_id = str(page_id).encode('utf-8')
	sha16 =sha1(page_id).hexdigest()
	sha10 =int(sha16, 16)

	chars =[]
	alphabets = string.digits +string.ascii_lowercase
	while sha10>0:
		sha10, r = divmod(sha10, 36)
		chars.append(alphabets[r])
	
	return ''.join(reversed(chars))

def writePage(title, wikiText, fobj):
	global user_id, username

	pglen = len(wikiText)
	time =datetime.now().strftime("%Y-%m-%dT%H-%M-%SZ")
	
	curPage ='''\n\n
	<page>
		<title>''' +title +'''</title>
		<ns>0</ns>
		<id>''' +str(page_id) +'''</id>
		<revision>
			<id>''' +str(page_id) +'''</id>
			<timestamp>'''+time+'''</timestamp>
			<contributor>
				<username>''' +username +'''</username>
				<id>''' +str(user_id) +'''</id>
			</contributor>
			<comment>xmlpage created</comment>
			<model>wikitext</model>
			<format>text/x-wiki</format>
			<text xml:space="preserve" bytes="''' +str(pglen) +'''">
			\n''' +wikiText +'''
			</text>
			<sha1>''' +sha36(page_id) +'''</sha1>
		</revision>
	</page>
	\n\n'''

	fobj.write(curPage)
	return


def getData(row):
  title = row['name_x']
  data = row.to_dict('index')
  print(data[0])
  return data[0]


file_loader = FileSystemLoader('')
env = Environment(loader=file_loader)
template = env.get_template('airport_template.j2')
data = pd.read_excel('sample.xlsx').astype('str') 
ids = data['name_x_english'].tolist()
print(ids)
fobj = open('sample.xml', 'w')
fobj.write(tewiki+'\n')

for i, Id in enumerate(ids):
  row = data.loc[data['name_x_english']==Id]
  title = row['name_x_english'].values[0]

  text =template.render(getData(row))
  print(text)
  writePage(title, text, fobj)
  print('\n', i, title)

fobj.write('</mediawiki>')
fobj.close()




['Sofia Airport']
{'ID': '3977', 'ident': 'LBSF', 'type_x': 'పెద్ద విమానాశ్రయం', 'name_x_english': 'Sofia Airport', 'name_x': 'సోఫియా విమానాశ్రయం', 'latitude_deg_x': '42.69669342', 'longitude_deg_x': '23.41143608', 'elevation_ft_x': '1742', 'continent': 'యూరప్', 'iso_country_code': 'BG', 'country_name': 'బల్గేరియా', 'iso_region': 'BG-23', 'iso_region_name': 'సోఫియా ప్రావిన్స్', 'municipality': 'సోఫియా', 'scheduled_service': 'yes', 'icao_code': 'LBSF', 'iata_code': 'SOF', 'local_code': 'nan', 'home_link': 'http://sofia-airport.bg', 'wikipedia_link': 'https://en.wikipedia.org/wiki/Sofia_Airport', 'airport_type': 'ప్రజా', 'operator_owner': 'nan', 'serves': 'సోఫియా', 'opened': '1937-09-16', 'hub_for': ' ఎఎల్కె ఎయిర్\u200cలైన్స్; బిహెచ్ ఎయిర్; బల్గేరియా ఎయిర్; యూరోపియన్ ఎయిర్ చార్టర్; గుల్లివ్ ఎయిర్; ర్యానైర్; విజ్ ఎయిర్', 'passengers': '3 364 151 ', 'aircraft_movements': '40 771 ', 'land_area': 'nan', 'owner': 'రాష్ట్ర యాజమాన్యం', 'operator': '\nSOF కనెక్ట్ AD', 'type_freq_eng': 'APP', 'ty